In [1]:
import torch
import numpy as np
from torch import Tensor
from dataclasses import dataclass
from pathlib import Path
from rdkit.Chem import GetPeriodicTable
torch.set_default_dtype(torch.float64)

ptable = GetPeriodicTable()

In [2]:
# Conversion module in Python

# Conversion factors
# Convert Bohr (a.u.) to Ångström and back
autoaa = 0.52917726  # Bohr to Ångström
aatoau = 1.0 / autoaa  # Ångström to Bohr

# Convert Hartree to eV and back
autoev = 27.21138505  # Hartree to eV
evtoau = 1.0 / autoev  # eV to Hartree

# Convert Hartree to kcal/mol and back
autokcal = 627.50947428  # Hartree to kcal/mol
kcaltoau = 1.0 / autokcal  # kcal/mol to Hartree

# Convert Hartree to kJ/mol and back
autokj = 2625.49964038  # Hartree to kJ/mol
kjtoau = 1.0 / autokj  # kJ/mol to Hartree

# Convert Hartree to reciprocal centimeters/wavenumbers and back
autorcm = 219474.63067  # Hartree to reciprocal centimeters (wavenumbers)
autowav = autorcm  # Alias for wavenumbers
rcmtoau = 1.0 / autorcm  # Reciprocal centimeters to Hartree
wavtoau = 1.0 / autowav  # Wavenumbers to Hartree

# Convert Hartree to nanometers and back
autonm = 45.56335266  # Hartree to nanometers
nmtoau = 1.0 / autonm  # Nanometers to Hartree

# Masses
# amu -> kg: conversion from atomic mass units to kg
amutokg = 1.660539040e-27  # Atomic mass units to kg
kgtoamu = 1.0 / amutokg  # kg to atomic mass units

# me -> kg: electron mass (a.u.) in kg
metokg = 9.10938356e-31  # Electron mass in kg
kgtome = 1.0 / metokg  # kg to electron mass

# amu -> au: conversion from a.u. to amu
amutoau = amutokg * kgtome  # Atomic mass units to atomic units
autoamu = kgtoamu * metokg  # Atomic units to atomic mass units

# Femtoseconds to atomic time units
fstoau = 41.3413733365614  # Femtoseconds to atomic units
autofs = 1.0 / fstoau  # Atomic units to femtoseconds

# Coulomb to atomic charge units (electrons)
autoc = 1.6021766208e-19  # Coulomb to atomic charge units
ctoau = 1.0 / autoc  # Atomic charge units to Coulomb

# Debye to atomic units
autod = autoc * 299792458 * autoaa**2 * fstoau * 1.0e+16  # Debye to atomic units
dtoau = 1.0 / autod  # Atomic units to Debye

# Convert meters/second to atomic units
mstoau = 1.0 / 2.18769126364e+06  # Meter/second to atomic units


atomic_mass_nist = [
    1.00794075,  4.00260193,  6.94003660,  9.01218307,
    10.81102805, 12.01073590, 14.00670321, 15.99940492,
    18.99840316, 20.18004638, 22.98976928, 24.30505162,
    26.98153853, 28.08549871, 30.97376200, 32.06478741,
    35.45293758, 39.94779856, 39.09830091, 40.07802251,
    44.95590828, 47.86674496, 50.94146504, 51.99613176,
    54.93804391, 55.84514443, 58.93319429, 58.69334711,
    63.54603995, 65.37778253, 69.72306607, 72.62755016,
    74.92159457, 78.95938856, 79.90352778, 83.79800000,
    85.46766360, 87.61664447, 88.90584030, 91.22364160,
    92.90637300, 95.95978854, 97.90721240, 101.06494014,
]
symbols = [ptable.GetElementSymbol(e+1) for e in range(len(atomic_mass_nist))]
atomic_mass_nist = dict(zip(symbols, atomic_mass_nist))
atomic_mass_nist

{'H': 1.00794075,
 'He': 4.00260193,
 'Li': 6.9400366,
 'Be': 9.01218307,
 'B': 10.81102805,
 'C': 12.0107359,
 'N': 14.00670321,
 'O': 15.99940492,
 'F': 18.99840316,
 'Ne': 20.18004638,
 'Na': 22.98976928,
 'Mg': 24.30505162,
 'Al': 26.98153853,
 'Si': 28.08549871,
 'P': 30.973762,
 'S': 32.06478741,
 'Cl': 35.45293758,
 'Ar': 39.94779856,
 'K': 39.09830091,
 'Ca': 40.07802251,
 'Sc': 44.95590828,
 'Ti': 47.86674496,
 'V': 50.94146504,
 'Cr': 51.99613176,
 'Mn': 54.93804391,
 'Fe': 55.84514443,
 'Co': 58.93319429,
 'Ni': 58.69334711,
 'Cu': 63.54603995,
 'Zn': 65.37778253,
 'Ga': 69.72306607,
 'Ge': 72.62755016,
 'As': 74.92159457,
 'Se': 78.95938856,
 'Br': 79.90352778,
 'Kr': 83.798,
 'Rb': 85.4676636,
 'Sr': 87.61664447,
 'Y': 88.9058403,
 'Zr': 91.2236416,
 'Nb': 92.906373,
 'Mo': 95.95978854,
 'Tc': 97.9072124,
 'Ru': 101.06494014}

In [3]:
mchrg_prod = 3 # 3 Means CID
prog = 8 # 8 means GFN2-xTB
nuc = 14



In [4]:

#   ! MS method
#   method = 0
#   ! logical for too small fragments
#   small=.false.
#   littlemass=.false.
#   ! no checking of input etc
#   check=.false.
#   ! more infos
#   verbose = .false.
#   ! no production
#   prod   =.false.
#   ! no equilibration
#   noeq   =.true.
#   ! test calls
#   eonly  =.false.
#   eonly0 =.false.
#   eonly1 =.false.
#   ! uniform velocity scaling
#   unity=.false.
#   ! check if cid was OK
#   stopcid = .false.
#   starting_md=.false.
#   No_ESI = .false.
#   ! HS-UHF ini only for frag runs
#   iniok  =.true.
#   ! dump every dumpstep MD steps for MOLDEN movie (=4 fs as default)
#   dumpstep=4
#   ! counts the number of QC calls
#   calls=0
#   ! set scaling temp to 0
#   tscale = 0.0_wp

#   ! GBSA Solvation Model
#   !solvent='none'
#   !gsolvstate=0

#   new_velo = 0.0d0
#   ! total MD time including ion tracks
#   ttime=0
#   dtime = 0.0d0
#   ! undefined spin state
#   mspin=0
#   ! neutral M
#   mchrg=0
#   ! is fragmented?
#   fragstate=0

#   t1 = 0.0_wp
#   t2 = 0.0_wp
#   w1 = 0.0_wp
#   w2 = 0.0_wp

# MS Method
method = 0
# If too small, no fragmentation can happen
small = False
littlemass = False

# Skip checking input
check = False

verbose = False


prod = False # Do production runs?
noeq = True # Do equilibration
eonly = False # test calls?
eonly0 = False
eonly1 = False

unity = False # uniform velocity scaling
stopcid = False # check if cid was OK
starting_md = False
No_ESI = False # Do electrospray ionization
iniok = True # HS-UHF initialization for fragmentation runs only
dumpstep = 4 # Dump molden data steps
calls = 0 # Count number of QC calls
tscale = 0.0 # Set temperature scale

new_velo = 0.0
ttime = 0 # Total MD time including ion tracks
dtime = 0.0 # Time step size

mspin = 0 # Molecule spin?
mchrg = 0 # Molecule charge
fragstate = 0 # Is fragmented

t1 = 0.0 # Timing variables (start, end)
t2 = 0.0
w1 = 0.0
w2 = 0.0

In [5]:
from dataclasses import dataclass, field
import torch

@dataclass
class RunSettings:
    tadd: float
    eimp: float
    xyzr: torch.Tensor = field(default_factory=lambda: torch.empty(0, 0, 0))
    velor: torch.Tensor = field(default_factory=lambda: torch.empty(0, 0, 0))
    eimpr: torch.Tensor = field(default_factory=lambda: torch.empty(0))
    taddr: torch.Tensor = field(default_factory=lambda: torch.empty(0))
    velofr: torch.Tensor = field(default_factory=lambda: torch.empty(0, 0))

@dataclass
class CollisionType:
    set_coll: int
    max_coll: int
    
    from dataclasses import dataclass, field

@dataclass
class StructureType:
    nat: int = 0
    nid: int = 0
    nbd: int = 0
    id: torch.Tensor = field(default_factory=lambda: torch.empty(0, dtype=torch.int32))
    num: torch.Tensor = field(default_factory=lambda: torch.empty(0, dtype=torch.int32))
    sym: list = field(default_factory=list)
    xyz: torch.Tensor = field(default_factory=lambda: torch.empty((0, 0), dtype=torch.float32))
    uhf: int = 0
    charge: float = 0.0
    lattice: torch.Tensor = field(default_factory=lambda: torch.empty((0, 0), dtype=torch.float32))
    periodic: torch.Tensor = field(default_factory=lambda: torch.empty(0, dtype=torch.bool))
    bond: torch.Tensor = field(default_factory=lambda: torch.empty((0, 0), dtype=torch.int32))
    comment: str = ""
    info: dict = field(default_factory=dict)
    sdf: list = field(default_factory=list)
    pdb: list = field(default_factory=list)

In [6]:
collauto = False
fullauto = False
manual = True
temprun = False

In [7]:
@dataclass
class Molecule:
    nat: int
    xyz: Tensor
    elem: list[str]
    iat: list[int]
    mass: Tensor
        
    @staticmethod
    def from_xyz(path: Path):
        nat, *arr = Path(path).read_text().split()
        nat = int(nat)
        arr = np.array(arr).reshape(-1,4)
        elem, xyz = arr[:,0], arr[:,1:]
        xyz = xyz.astype(float)
        iat = torch.tensor([ptable.GetAtomicNumber(symbol) for symbol in elem])
        mass = torch.tensor([atomic_mass_nist[symbol] for symbol in elem])
        
        assert nat == len(xyz), f'nat == {nat} while len(xyz) == {len(xyz)}'
        return Molecule(
            nat = nat,
            xyz=xyz,
            elem=elem.tolist(),
            iat=iat,
            mass=mass,
        )
mol_file = '../runs/current/TMPQCXMS/TMP.1/start.xyz'
coord_file = '../runs/current/TMPQCXMS/TMP.1/'
# mol = 
# !cat {mol_file}
mol = Molecule.from_xyz(mol_file)

In [8]:
# This means we have transition metals or s4/5s elems
ECP = False

In [9]:
nuc = mol.nat
ntraj = nuc * 25
ntraj

350

In [10]:
ax: float = 0 # 

def setetemp(nfrag: int, eimp:float) -> float:
    """
    Estimate the electronic temp in Fermi smearing for given IEE and ax
    """
    etemp =  5000. + 20000. * ax
    if eimp > 0 and nfrag <= 1:
        tmp = max(eimp, 0)
        etemp = etemp + tmp * ieetemp
        
    return etemp


ieetemp: float = 0 # TODO: Decode 'relate IEE to eTemp in SCF'
betemp: float = 0 # 
betemp = setetemp(nfrag=1, eimp=-1.0)

In [11]:
iseed: int = 0
torch.manual_seed(iseed)
randx = torch.rand(1,)
randx

tensor([0.9701])

In [12]:
nmax0 = ntraj * 50
nmax0

17500

In [13]:
betemp = 5000.0 # TODO: Any effects from this? "Hardcoded for xTB for now"

In [14]:
tmax = 5 # Max running time in picoseconds
tmax = tmax * 1000.0 # Max running time in femtoseconds
tstep = 0.5 # Timestep in femtoseconds
nmax = tmax / tstep # Total steps
tmax, nmax, tstep

(5000.0, 10000.0, 0.5)

In [15]:
# Convert femtoseconds to atomic units (aus)


eimp0 = 70 # Electronvolts
eimp0 = eimp0 * evtoau # energy in atomic units
tstep, eimp0

(0.5, 2.572452665359642)

In [16]:
# Are there any 4s-4d/5s-5d elements?
noecp = False
ECP = False

In [17]:
mol.iat

tensor([8, 6, 6, 6, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [18]:
nuc

14

In [19]:
mass = mol.mass * amutoau
mass

tensor([29165.1310, 21894.2322, 21894.2322, 21894.2322, 21894.2322,  1837.3636,
         1837.3636,  1837.3636,  1837.3636,  1837.3636,  1837.3636,  1837.3636,
         1837.3636,  1837.3636])

In [20]:
method = 3 # 3 means CID
# TODO 
assert prog != 0, "CID is not suitable with DFTB, why?"

In [21]:
mass

tensor([29165.1310, 21894.2322, 21894.2322, 21894.2322, 21894.2322,  1837.3636,
         1837.3636,  1837.3636,  1837.3636,  1837.3636,  1837.3636,  1837.3636,
         1837.3636,  1837.3636])

In [22]:
velof = 1.0

# Set charge
mchrg_prod = 1
mchrg = mchrg_prod
chrgcont = mchrg

collisions = 0
edum = 0

# cidcheck():
@dataclass
class CollisionType:
    max_coll: int = 6
    set_coll: int = 10
    
coll = CollisionType()

In [23]:
tadd = 0.0
eimp = 0.0
velof = 1.0
# Smaller than 7 atoms doesn't work
# 
small = nuc <= 7
assert not small

In [24]:
starting_md = False

In [25]:
manual = True

In [26]:
# when collsec > 0, fragment until you have collsec fragments
# specifically. First fragmentation can yield max collsec[0] fragments
# second, max collsec[1], etc
collsec = torch.zeros(3)
# when collno > 0, force this many collisions
collno = torch.zeros(3)


ELAB = 40 # eV
maxcoll = 6  # Do this many collisions max

if coll.max_coll != 0:
    print('!!! M+ collisions are user set !!!')
    collisions = coll.max_coll
    new_counter = 1 # This counts total fragments at current step

!!! M+ collisions are user set !!!


### Start Loop for CID collision and subsequent MD simulation

In [27]:
itrj: int = 0
icoll: int = 0
isec: int = 0

In [28]:
isec = 1
icoll = icoll + 1
fragstate = 0

In [29]:
itraj = 1
collisions = 6
mchrg = 1
chrgcont = 1.0

In [30]:
# direc(3),cm(3),cm1(3),cm2(3)
direc, cm, cm1, cm2 = torch.zeros((4, 3), dtype=torch.float32)

if icoll != 1:
    direc = cm2 - cm1
    direc /= torch.norm(direc, p=2)

In [31]:
direc

tensor([0., 0., 0.], dtype=torch.float32)

### CID

In [32]:
@dataclass
class GasParameters:
    indatom: int
    iatom: int
    "Is this the mass of atom"
    miatom: float
    riatom: float
    
gas = GasParameters(
    indatom=18,
    iatom=3,
    # IMO This is mass of the collision gas atom
    miatom=72820.752605496193, # TODO: What are these?
    riatom=3.5526664257049561,
)

In [33]:
nuc0 = 0
# This means: 
# If gas.iatom not 6, then we have noble bas and we add 1 more atom
# into system. Else, it's a 6, it's diatomic N2, and we add 2 atoms to system
if gas.iatom != 6: 
    nuc0 = nuc + 1
if gas.iatom == 6:
    nuc0 = nuc + 2
 
#   allocate(xyz0(3,nuc0), &
#       velo0(3,nuc0), &
#       grad0(3,nuc0), &
#       mass0(nuc0), &
#       iat0(nuc0), &
#       achrg0(nuc0), &
#       aspin0(nuc0)) 

xyz0 = torch.zeros((3, nuc0))
velo0 = torch.zeros((3, nuc0))
grad0 = torch.zeros((3, nuc0))
mass0 = torch.zeros(nuc0)
iat0 = torch.zeros(nuc0)
achrg0 = torch.zeros(nuc0)
aspin0 = torch.zeros(nuc0)

In [34]:
!cat ../runs/current/TMPQCXMS/TMP.1/rotate.xyz

          14
  
 O   -0.86135082211542224         -7.7682449169200414          15.461982204809336     
 C    0.10274853878796679         -8.0728695108798547          14.510603513804643     
 C   -0.47484132835964982         -8.9397518228940616          13.461802760056679     
 C    -1.8586955962086826         -9.3937382860206586          13.960370290629234     
 C    -1.8814275696019482         -8.7268087767610254          15.385819309309317     
 H    0.55607803892982255         -7.1547427313617762          14.243171614386661     
 H    0.75506858575479940         -8.7522352149267650          15.002049905989383     
 H     9.2617440145273303E-002    -9.8084428627829396          13.340438601361978     
 H   -0.57239447889376383         -8.3569837234193329          12.477135115157155     
 H    -1.8892657431770707         -10.403167193270098          14.228212831600977     
 H    -2.7243216943253628         -9.1360490691365435          13.439273567389437     
 H    -1.7231648860042426  

In [35]:
!du -hs ../runs/current/TMPQCXMS/TMP.1/rotate.xyz
!cat ../runs/current/TMPQCXMS/TMP.1/rotate.xyz

4.0K	../runs/current/TMPQCXMS/TMP.1/rotate.xyz
          14
  
 O   -0.86135082211542224         -7.7682449169200414          15.461982204809336     
 C    0.10274853878796679         -8.0728695108798547          14.510603513804643     
 C   -0.47484132835964982         -8.9397518228940616          13.461802760056679     
 C    -1.8586955962086826         -9.3937382860206586          13.960370290629234     
 C    -1.8814275696019482         -8.7268087767610254          15.385819309309317     
 H    0.55607803892982255         -7.1547427313617762          14.243171614386661     
 H    0.75506858575479940         -8.7522352149267650          15.002049905989383     
 H     9.2617440145273303E-002    -9.8084428627829396          13.340438601361978     
 H   -0.57239447889376383         -8.3569837234193329          12.477135115157155     
 H    -1.8892657431770707         -10.403167193270098          14.228212831600977     
 H    -2.7243216943253628         -9.1360490691365435          13.4

In [36]:
!cat ../runs/current/TMPQCXMS/TMP.1/rotate.xyz
!ls ../runs/current/TMPQCXMS/TMP.1/CID.xyz

          14
  
 O   -0.86135082211542224         -7.7682449169200414          15.461982204809336     
 C    0.10274853878796679         -8.0728695108798547          14.510603513804643     
 C   -0.47484132835964982         -8.9397518228940616          13.461802760056679     
 C    -1.8586955962086826         -9.3937382860206586          13.960370290629234     
 C    -1.8814275696019482         -8.7268087767610254          15.385819309309317     
 H    0.55607803892982255         -7.1547427313617762          14.243171614386661     
 H    0.75506858575479940         -8.7522352149267650          15.002049905989383     
 H     9.2617440145273303E-002    -9.8084428627829396          13.340438601361978     
 H   -0.57239447889376383         -8.3569837234193329          12.477135115157155     
 H    -1.8892657431770707         -10.403167193270098          14.228212831600977     
 H    -2.7243216943253628         -9.1360490691365435          13.439273567389437     
 H    -1.7231648860042426  

In [37]:
fname = 'CID.xyz'
iniok      = True
stopcid    = False
gradfail   = False
avg_struc =  False
dumpscreen = 100   #interval for screen dumping
dumpcoord  = 4     #interval for coordinate dumping
dumpdist   = 10    #interval for distance dumping
dumpavg    = 50    #
ntot       = 15000 #maximum number of steps

new_velo   = 0.0
Tinit      = 0.0

time_step_count = 0
step_counter = 0

avxyz = 0
avxyz2 = 0

normmass =0
cg = 0
diff_cg = 0
cnt = 0
trafo = 0
gradient = 0

set_grad = float('+inf')

T_GBSA=300
solvent='h2o'
gsolvstate=0
nfrag = 1
# count steps after frag
morestep = 0
check_fragmented = 1
start_cnt = 0
# count the number of steps that are used for 
# averaging structures for IP calc
cnt_steps = 50

In [38]:
# abs charge higher than 1 could need more sim steps, who knows?
#   !if ( abs(mchrg) == 1) cnt_steps = 50
#   !if ( abs(mchrg) > 1) cnt_steps = 100
# this is not sufficiently tested. It was experienced that
# higher charge (or maybe larger molecules)
# need more time for rearrangement and this can influence the 
# IP assignment in the end
if nuc <= 10:
    add_steps = 0
if nuc > 10:
    add_steps = (nuc/10) * 500
if nuc > 10:
    add_steps = (nuc/10) * 1000
print(add_steps)

1400.0


In [39]:
# set eletronic temperature
etemp = -1
if etemp <= 0: # Fermi-smearing levels
    etemp = 5000.0   # + 20000.0
etemp

5000.0

In [40]:
cm = torch.tensor([-1.8763399533567406, -16.157524061279283, 27.574184314951335])
cm

tensor([ -1.8763, -16.1575,  27.5742])

In [41]:
mol.xyz.shape

(14, 3)

In [42]:
mol.xyz # TODO somewhere the xyz changed


array([[ -0.86135074,  -7.7682442 ,  15.46198077],
       [  0.10274853,  -8.07286876,  14.51060217],
       [ -0.47484128,  -8.93975099,  13.46180151],
       [ -1.85869542,  -9.39373741,  13.960369  ],
       [ -1.8814274 ,  -8.72680797,  15.38581788],
       [  0.55607799,  -7.15474207,  14.24317029],
       [  0.75506852,  -8.7522344 ,  15.00204851],
       [  0.09261743,  -9.80844195,  13.34043736],
       [ -0.57239443,  -8.35698295,  12.47713396],
       [ -1.88926557, -10.40316623,  14.22821151],
       [ -2.72432144,  -9.13604822,  13.43927232],
       [ -1.72316473,  -9.53070382,  16.13942656],
       [ -2.82500974,  -8.25325416,  15.73297612],
       [ -1.019431  ,  -6.85785249,  15.3938455 ]])

In [43]:
# velo, xyz
summass = mass.sum()
summass 

tensor(133278.3320)

In [44]:
gas.miatom

72820.75260549619

In [45]:
Eimpact = ELAB # IMO This is initial energy. it gets transferred
# to ECOM, after first iter
ECOM = 0

In [46]:
eExact = False
def vary_energies(E_in, E_Distr) -> float:
    pass

if not eExact:
    E_Distr = .1
    E_velo = 1.8379383706653682 #vary_energies(Eimpact, E_Distr) * evtoau

In [47]:
Ekin = 0.027652043145031752
Tinit = 415.80156382302368

kB = 3.166808578545117e-06 # Boltzmann const in Eh/K
beta = gas.miatom / (gas.miatom + summass)
E_Therm = Tinit * 0.5 * 3.0 * nuc * kB

E_COM_calc = (beta * E_velo) * autoev # Calculate the Center-of-mass energy
E_COM_calc

tensor(17.6710)

In [63]:
from math import sqrt
fasti = sqrt(2 * E_velo / summass)
fasti

0.005251709510040625

In [66]:
## BEGIN CID MODULE

print(f'Collision Gas idx {gas.indatom}') # toSymbol(gas%IndAtom)
print(f'Number of precursor atoms {nuc}')
print(f'Mass collision gas {gas.miatom/amutoau}')
print(f'Mass precursor Ion {summass * autoamu}')
print()
print(f'Kinetic Energy eV (LAB) {E_velo / evtoau}')
print(f'Kinetic Energy au (LAB) {E_velo}')
print(f'Kinetic Energy eV (COM) {E_COM_calc}')
print(f'Kinetic Energy au (COM) {E_COM_calc * evtoau}')
print()
print(f'Velocity (au) ', fasti)
print(f'Velocity (au) ', fasti / mstoau)

Collision Gas idx 18
Number of precursor atoms 14
Mass collision gas 39.948001861572266
Mass precursor Ion 73.11381526999998

Kinetic Energy eV (LAB) 50.012848702344954
Kinetic Energy au (LAB) 1.8379383706653682
Kinetic Energy eV (COM) 17.67098233295506
Kinetic Energy au (COM) 0.6493966514561912

Velocity (au)  0.005251709510040625
Velocity (au)  11489.119014290982


In [75]:
E_rot = 0.0020249391061874212 # rotation_velo(xyz, nuc, mass, velo, velo_rot, E_rot)

print('-'*10, 'Molecular Ion Properties', '-'*10)
print(f'Molecular temp. (K) {Tinit}')
print(f'Rotational Energy (ev) {E_rot / evtoau}')
print(f'Thermal Energy (eV) {E_Therm / evtoau}')
print(f'Internal Energy (eV) {(E_rot+E_Therm) / evtoau}')

---------- Molecular Ion Properties ----------
Molecular temp. (K) 415.8015638230237
Rotational Energy (ev) 0.05510139772126875
Thermal Energy (eV) 0.752450393438672
Internal Energy (eV) 0.8075517911599408


In [54]:
E_velo

1.8379383706653682